In [10]:
import os
import os.path
import numpy as np
import h5py
import json
import torch
import imageio                           # is used as like scipy 'imread'
import scipy
from scipy.misc import imread,imresize
from PIL import Image
from tqdm import tqdm
from collections import Counter
from random import seed, choice, sample
import string
import numpy

In [11]:
# initialization for the dataset
dataset='coco'
json_path='coco_raw.json'
image_folder='/home/hemanth/Documents/dataset/MS_COCO_2017'
captions_per_image=5
threshold_freq=5
output_folder=''

In [12]:
with open(json_path, 'r') as j:
    imgs = json.load(j)

In [13]:
def prepro_captions(imgs):
    # preprocess all the caption
    print('example processed tokens:')
    for i,img in enumerate(imgs):
        img['processed_tokens'] = []
        for j,s in enumerate(img['captions']):
            if i < 5 and j == 0: print('before : ',str(s))
            txt = str(s).lower().translate(string.punctuation).strip("").split()    #translate(None, string.punctuation)
            img['processed_tokens'].append(txt)
            if i < 5 and j == 0: print('after : ',txt,'\n')
    print("\n\n")
# imgs ={ {filepath:, id: ,captions:{},processed_tokens:[]}, {filepath:, id: ,captions:{},processed_tokens:[]}, ....}
    print(imgs[0],'\n')

# tokenization and preprocessing 
prepro_captions(imgs)

example processed tokens:
before :  A man is in a kitchen making pizzas.
after :  ['a', 'man', 'is', 'in', 'a', 'kitchen', 'making', 'pizzas.'] 

before :  The dining table near the kitchen has a bowl of fruit on it.
after :  ['the', 'dining', 'table', 'near', 'the', 'kitchen', 'has', 'a', 'bowl', 'of', 'fruit', 'on', 'it.'] 

before :  a person with a shopping cart on a city street 
after :  ['a', 'person', 'with', 'a', 'shopping', 'cart', 'on', 'a', 'city', 'street'] 

before :  A person on a skateboard and bike at a skate park.
after :  ['a', 'person', 'on', 'a', 'skateboard', 'and', 'bike', 'at', 'a', 'skate', 'park.'] 

before :  a blue bike parked on a side walk 
after :  ['a', 'blue', 'bike', 'parked', 'on', 'a', 'side', 'walk'] 




{'file_path': '/home/hemanth/Documents/dataset/MS_COCO_2017/val2017/000000397133.jpg', 'id': 397133, 'captions': ['A man is in a kitchen making pizzas.', 'Man in apron standing on front of oven with pans and bakeware', 'A baker is working in the kit

In [14]:
t=0
v=0
for i,img in enumerate(imgs):
    if img['file_path'][45] == 't':
        t=t+1
    else:
        v=v+1
print("train : %d"%t, "val : %d"%v)
print("\nActual Total no of captions:",5*(t+v))

train : 118287 val : 5000

Actual Total no of captions: 616435


In [15]:
# max length?
# word frequency?
sent_lengths = {}
word_freq=Counter()
totcap=0
for img in imgs:
    for txt in img['processed_tokens']:
        totcap=totcap+1
        word_freq.update(txt)
        nw = len(txt)
        sent_lengths[nw] = sent_lengths.get(nw, 0) + 1

max_len = max(sent_lengths.keys())
print('Calculated & processed total no of captions :  %d' % totcap)
print('max length sentence in raw data :  %d\n' % max_len)
print('sentence length distribution (count, number of words):')
sum_len = sum(sent_lengths.values())
for i in range(max_len+1):
    print('%2d: %10d   %f%%' % (i, sent_lengths.get(i,0), sent_lengths.get(i,0)*100.0/sum_len))
print("\nWord Frequencies were counted successfully..!!")
#print('\nWord frequencies:\n\n',word_freq)

Calculated & processed total no of captions :  616435
max length sentence in raw data :  50

sentence length distribution (count, number of words):
 0:          0   0.000000%
 1:          0   0.000000%
 2:          0   0.000000%
 3:          0   0.000000%
 4:          0   0.000000%
 5:          1   0.000162%
 6:         13   0.002109%
 7:       4797   0.778184%
 8:     102041   16.553408%
 9:     134485   21.816574%
10:     132520   21.497806%
11:      95052   15.419631%
12:      60408   9.799573%
13:      35064   5.688191%
14:      19947   3.235864%
15:      11405   1.850155%
16:       6846   1.110579%
17:       4274   0.693342%
18:       2733   0.443356%
19:       1899   0.308062%
20:       1301   0.211052%
21:        910   0.147623%
22:        665   0.107878%
23:        500   0.081112%
24:        326   0.052885%
25:        261   0.042340%
26:        180   0.029200%
27:        164   0.026605%
28:         88   0.014276%
29:         71   0.011518%
30:         60   0.009733%
31:        

In [16]:
# thresholding the words
# creating word map or vocab mapping  'word to index' and 'index to word'
vocab=[w for w in list(word_freq.keys()) if word_freq[w] > threshold_freq]

w2i = {k: v + 1 for v, k in enumerate(vocab)}
w2i['<unk>'] = len(w2i) + 1
w2i['<start>'] = len(w2i) + 1
w2i['<end>'] = len(w2i) + 1
w2i['<pad>'] = 0

i2w = {v + 1: k for v, k in enumerate(vocab)}
i2w[len(i2w) + 1] = '<unk>'
i2w[len(i2w) + 1] = '<start>'
i2w[len(i2w) + 1] = '<end>'
i2w[0]='<pad>'
# saving the 'w2i and i2w' dictionary by json file format
with open('w2i_coco.json','w') as j:
    json.dump(w2i, j)
with open('i2w_coco.json','w') as j:
    json.dump(i2w,j)
print('"Word - Index" mapping is done!!..\n')
print('Vocab size: ',len(vocab))

"Word - Index" mapping is done!!..

Vocab size:  13027


In [8]:
imgs[0]['file_path']

'/home/hemanth/Documents/dataset/MS_COCO_2017/val2017/000000397133.jpg'

In [10]:
print("train and val dataset splitting...")
# spliting the train and val datasets seperately..  
train_image_paths=[]
train_image_captions=[]
val_image_paths=[]
val_image_captions=[]
for img in tqdm(imgs,position=0,leave=True):
    impath = img['file_path']
    imcaps = img['processed_tokens']
    if img['file_path'][45] == 't':
        train_image_paths.append(impath)
        train_image_captions.append(imcaps)
    elif img['file_path'][45] == 'v':
        val_image_paths.append(impath)
        val_image_captions.append(imcaps)
    else:
        print('ERROR............in assigining dataset paths...........!!!!!!!!!')

100%|██████████| 123287/123287 [00:00<00:00, 1608653.13it/s]

train and val dataset splitting...


In [12]:
base_filename = dataset + '_' + str(captions_per_image) + '_cap_per_img_' + str(threshold_freq) + '_min_word_freq'
seed(123)
for impaths, imcaps, split in [(train_image_paths, train_image_captions, 'TRAIN'),
                                   (val_image_paths, val_image_captions, 'VAL')]:

    # removing the same file if already existing..
    rem_file=os.path.join(output_folder, split + '_IMAGES_' + base_filename + '.hdf5')
    if os.path.exists(rem_file):
        os.remove(rem_file)
    # saving in hdf5 file
    print("HDF5 file creation for %s started...!!!"%split)
    with h5py.File(os.path.join(output_folder, split + '_IMAGES_' + base_filename + '.hdf5'), 'a') as h:
        h.attrs['captions_per_image'] = captions_per_image

        # Create dataset inside HDF5 file to store images
        images = h.create_dataset('images', (len(impaths), 3, 256, 256), dtype='uint8')

        print("\nReading %s images and captions, storing to file...\n" % split)

        enc_captions = []
        caplens = []

        for i, path in enumerate(tqdm(impaths,position=0,leave=True)):

            # Sample captions
            if len(imcaps[i]) < captions_per_image:
                captions = imcaps[i] + [choice(imcaps[i]) for _ in range(captions_per_image - len(imcaps[i]))]
            else:
                captions = sample(imcaps[i], k=captions_per_image)

            # Sanity check
            assert len(captions) == captions_per_image

            # Read images
            #from skimage import io         im = io.imread(self.file_image)
            img = imread(impaths[i])
            if len(img.shape) == 2:
                img = img[:, :, np.newaxis]
                img = np.concatenate([img, img, img], axis=2)
            img = imresize(img, (256, 256))
            img = img.transpose(2, 0, 1)
            assert img.shape == (3, 256, 256)
            assert np.max(img) <= 255

            # Save image to HDF5 file
            images[i] = img

            for j, c in enumerate(captions):
                # Encode captions
                enc_c = [w2i['<start>']] + [w2i.get(word, w2i['<unk>']) for word in c] + [
                        w2i['<end>']] + [w2i['<pad>']] * (max_len - len(c))

                # Find caption lengths
                c_len = len(c) + 2

                enc_captions.append(enc_c)
                caplens.append(c_len)

        print("\nHDF5 file for %s created successfully.......!!!!\n"%split)
        
        # Sanity check
        assert images.shape[0] * captions_per_image == len(enc_captions) == len(caplens)
        # Save encoded captions and their lengths to JSON files
        with open(os.path.join(output_folder, split + '_CAPTIONS_' + base_filename + '.json'), 'w') as j:
                json.dump(enc_captions, j)

        with open(os.path.join(output_folder, split + '_CAPLENS_' + base_filename + '.json'), 'w') as j:
                json.dump(caplens, j)
        print("JSON file for %s is created"%split)
        
        print("\nALL JSON file created successfully.......!!!!\n")
        print("ALL Input files were preprocessed successfully...........!!!!!")

  0%|          | 0/118287 [00:00<?, ?it/s]/home/hemanth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/home/hemanth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  0%|          | 13/118287 [00:00<15:36, 126.26it/s]

HDF5 file creation for TRAIN started...!!!

Reading TRAIN images and captions, storing to file...



100%|██████████| 118287/118287 [47:26<00:00, 41.56it/s]  



HDF5 file for TRAIN created successfully.......!!!!

JSON file for TRAIN is created

ALL JSON file created successfully.......!!!!

ALL Input files were preprocessed successfully...........!!!!!
HDF5 file creation for VAL started...!!!

Reading VAL images and captions, storing to file...



100%|██████████| 5000/5000 [01:40<00:00, 49.77it/s]



HDF5 file for VAL created successfully.......!!!!

JSON file for VAL is created

ALL JSON file created successfully.......!!!!

ALL Input files were preprocessed successfully...........!!!!!
